#### -.-.- Cryptingup servis çağrısı sonucu oluşan json dosyasının şeması tanımlanır -.-.-

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType, TimestampType, ArrayType

In [0]:
asset_schema = StructType(fields=[StructField("exchange_id", StringType(), False), 
								  StructField("symbol", StringType(), False), 
								  StructField("base_asset", StringType(), False),
								  StructField("quote_asset", StringType(), False),
								  StructField("price_unconverted", FloatType(), True), 
								  StructField("price", FloatType(), False),
								  StructField("change_24h", FloatType(), False),
								  StructField("spread", FloatType(), True),
								  StructField("volume_24h", FloatType(), True),
								  StructField("status", StringType(), True),
								  StructField("created_at",TimestampType(), True),
								  StructField("updated_at", TimestampType(), False)
								  ]) 

In [0]:
getallassetsresp_schema = StructType(fields=[StructField("markets", ArrayType(asset_schema), True),
											 StructField("next", StringType(), True)
											 ])

#### -.-.- Şema verilerek dosya data frame'e alınır -.-.-

In [0]:
getallassets_df = spark.read.schema(getallassetsresp_schema).option("multiLine", True).json("/mnt/dataenggpprjdl/raw/cryptingupAllAssetsUSD.json")

In [0]:
# getallassets_df.printSchema()

root
-- markets: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- exchange_id: string (nullable = true)
 | |-- symbol: string (nullable = true)
 | |-- base_asset: string (nullable = true)
 | |-- quote_asset: string (nullable = true)
 | |-- price_unconverted: float (nullable = true)
 | |-- price: float (nullable = true)
 | |-- change_24h: float (nullable = true)
 | |-- spread: float (nullable = true)
 | |-- volume_24h: float (nullable = true)
 | |-- status: string (nullable = true)
 | |-- created_at: timestamp (nullable = true)
 | |-- updated_at: timestamp (nullable = true)
-- next: string (nullable = true)

#### -.-.- Veri tabanına kaydedilecek data frame oluşturulur -.-.-

In [0]:
from pyspark.sql.functions import current_timestamp, explode, col

In [0]:
asset_df = ( getallassets_df.select("*", explode("markets").alias("marketitem"))
                            .select("*", "marketitem.*")
                            .withColumn("ingested_at", current_timestamp())
                            .drop(col("next")) 
                            .drop(col("markets"))
                            .drop(col("marketitem"))
                            .drop(col("created_at"))
                            .drop(col("price_unconverted"))
                            )


In [0]:
# display(asset_df)

exchange_id,symbol,base_asset,quote_asset,price,change_24h,spread,volume_24h,status,updated_at,ingested_at
COINBASE,SHIB-USD,SHIB,USD,2.6335E-5,0.9390571,0.03796507,5.303864E8,recent,2021-10-10T15:52:45.551+0000,2021-10-10T19:11:08.298+0000
COINBASE,BTC-USD,BTC,USD,55364.355,0.67350113,0.006628594,4.18989024E8,recent,2021-10-10T15:52:46.368+0000,2021-10-10T19:11:08.298+0000
COINBASE,ETH-USD,ETH,USD,3554.7,-1.2958957,0.007876585,3.38278816E8,recent,2021-10-10T15:52:50.119+0000,2021-10-10T19:11:08.298+0000
BITFINEX,BTC-USD,BTC,USD,55379.0,0.44619375,0.0036114121,2.09885088E8,recent,2021-10-10T15:52:49.092+0000,2021-10-10T19:11:08.298+0000
COINBASE,SOL-USD,SOL,USD,154.148,-2.4330347,0.035025135,1.36130128E8,recent,2021-10-10T15:52:50.126+0000,2021-10-10T19:11:08.298+0000
COINBASE,XYO-USD,XYO,USD,0.0396815,26.305822,0.1888265,1.3288672E8,recent,2021-10-10T15:52:46.199+0000,2021-10-10T19:11:08.298+0000
COINBASE,DOT-USD,DOT,USD,36.0935,1.9072223,0.0027705436,1.24206552E8,recent,2021-10-10T15:52:46.575+0000,2021-10-10T19:11:08.298+0000
COINBASE,ADA-USD,ADA,USD,2.24335,-1.1957718,0.022285612,1.06497616E8,recent,2021-10-10T15:52:50.118+0000,2021-10-10T19:11:08.298+0000
KRAKEN,XBT-USD,XBT,USD,55378.65,0.85605896,1.8057485E-4,7.9736088E7,recent,2021-10-10T15:52:45.563+0000,2021-10-10T19:11:08.298+0000
COINBASE,LINK-USD,LINK,USD,26.843065,-1.3061652,0.040486433,6.4822248E7,recent,2021-10-10T15:52:46.128+0000,2021-10-10T19:11:08.298+0000


#### -.-.- Veriler assets tablosuna append edilir, new_assets tablosu overwrite edilir -.-.-

In [0]:
asset_df.write.mode("append").format("delta").saveAsTable("db_processed.assets")

In [0]:
asset_df.write.mode("overwrite").format("delta").saveAsTable("db_processed.new_assets")